In [ ]:
import pickle 
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Hamoye Internship/Team_algorithm/CVSS_metrics_prediction/cve_merged.csv")
df.head()

**Load models**

In [ ]:
#Define f1_score function to load deep learning cvss metrics model
import keras.backend as K
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
#load cvss metrics model
from tensorflow.keras.models import load_model
path = "/content/drive/MyDrive/Hamoye Internship/Team_algorithm/CVSS_metrics_prediction/bert_multi_ouput_all.h5"
model = load_model(
       (path),
       custom_objects={'KerasLayer':hub.KerasLayer,"f1_score":f1_score})
#load cvss score model
with open('/content/drive/MyDrive/Hamoye Internship/Team_algorithm/CVSS_Prediction/cvss_model.pkl', 'rb') as handle:
    model_cvss = pickle.load(handle)

**Custom prediction function**

In [ ]:
def predict(Input_):
  input_text = [Input_]
  preds1 =  model.predict(input_text) #predict cvss metric
  class_index= []
  for i in  preds1:
    index = np.argmax(i)
    class_index.append(index)
  target_array = list(target_dic.values())
  targets = []
  for i,j in zip(target_array,class_index):
    val = i[j]
    targets.append(val)
  prediction_metric_cvss = {} # store as a dictionary
  for feature,label in zip(list(target_dic.keys()),targets):
    prediction_metric_cvss[feature] = label
  #one hot encode predicted metrics
  x = x = pd.DataFrame(predicted, index=range(1))

  x = pd.get_dummies(data=x[['access_authentication', 'access_vector', 'access_complexity', 
                    'impact_availability', 'impact_confidentiality', 'impact_integrity']], 
                    prefix=['access_authentication', 'access_vector', 'access_complexity', 'impact_availability',
                            'impact_confidentiality', 'impact_integrity'],)
  predicted_cols = x.columns
  feature_cols = ['access_authentication_MULTIPLE', 'access_authentication_NONE',
       'access_authentication_SINGLE', 'access_vector_ADJACENT_NETWORK',
       'access_vector_LOCAL', 'access_vector_NETWORK',
       'access_complexity_HIGH', 'access_complexity_LOW',
       'access_complexity_MEDIUM', 'impact_availability_COMPLETE',
       'impact_availability_NONE', 'impact_availability_PARTIAL',
       'impact_confidentiality_COMPLETE', 'impact_confidentiality_NONE',
       'impact_confidentiality_PARTIAL', 'impact_integrity_COMPLETE',
       'impact_integrity_NONE', 'impact_integrity_PARTIAL']
  dict_cvss_input = {}
  for f in feature_cols:
    if f not in predicted_cols:
      dict_cvss_input[f] = 0
    else:
      dict_cvss_input[f] = 1
  input_data =  pd.DataFrame(dict_cvss_input,index=range(1))
  cvss_score = np.round(model_cvss.predict(input_data),2)
  return prediction_metric_cvss,cvss_score